# Stellar Mass Black Hole Companion

## Background

Here we will look at how to detect a stellar mass black hole in a binary system! This tutorial is HEAVILY inspired by Adrian Price-Whelan's [tutorial](https://thejoker.readthedocs.io/en/latest/examples/Thompson-black-hole.html) (most of the code is from there!), but don't look there for the answers!

Here we will be trying to repoduce the discovery from [Thompson et al. 2019](https://ui.adsabs.harvard.edu/abs/2019Sci...366..637T/abstract), who found a a stellar mass black hole in a binary system with a giant star in our Milky Way. We will combine their TRES radial velocity data with data from APOGEE to determine the orbital parameters of the system, including the binary mass!

## Data

Below we will add the approproiate imports and the data needed for the tutorial:

In [ ]:
import arviz as az
import astropy.units as u
import corner
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import thejoker as tj
import thejoker.units as xu
from astropy.io import ascii
from astropy.time import Time
import os
%matplotlib inline
plt.style.use('../mystyle.mplstyle')

# set up a random number generator to ensure reproducibility
seed = 42
rnd = np.random.default_rng(seed=seed)

# data from Thompson et al; (2019)
tres_tbl = ascii.read(
    """8006.97517 0.000 0.075
    8023.98151 -43.313 0.075
    8039.89955 -27.963 0.045
    8051.98423 10.928 0.118
    8070.99556 43.782 0.075
    8099.80651 -30.033 0.054
    8106.91698 -42.872 0.135
    8112.81800 -44.863 0.088
    8123.79627 -25.810 0.115
    8136.59960 15.691 0.146
    8143.78352 34.281 0.087""",
    names=["HJD", "rv", "rv_err"],
)
tres_tbl["rv"].unit = u.km / u.s
tres_tbl["rv_err"].unit = u.km / u.s

# APOGEE data
apogee_tbl = ascii.read(
    """6204.95544 -37.417 0.011
    6229.92499 34.846 0.010
    6233.87715 42.567 0.010""",
    names=["HJD", "rv", "rv_err"],
)
apogee_tbl["rv"].unit = u.km / u.s
apogee_tbl["rv_err"].unit = u.km / u.s

To fit the orbit in this tutorial, we will be using [thejoker](https://thejoker.readthedocs.io/en/latest/index.html), which is a custom Monte Carlo sampler for the two-body problem. As a result, we will need to create some custom data objects that will be used by thejoker to do the sampling. This is done below.

In [ ]:
tres_data = tj.RVData(
    t=Time(tres_tbl["HJD"] + 2450000, format="jd", scale="tcb"),
    rv=u.Quantity(tres_tbl["rv"]),
    rv_err=u.Quantity(tres_tbl["rv_err"]),
)

apogee_data = tj.RVData(
    t=Time(apogee_tbl["HJD"] + 2450000, format="jd", scale="tcb"),
    rv=u.Quantity(apogee_tbl["rv"]),
    rv_err=u.Quantity(apogee_tbl["rv_err"]),
)

tres_data.plot(color=None, label="TRES")
plt.legend(fontsize=18)



## First, the TRES data

First, let us just fit the TRES data! Above we can see that the RV signal is periodic (yay!), but the errors are TINY (yes, they are there). thejoker allows you to add some non-linear parameters to the model, such as an extra uncertainty to the errors:
$$
\sigma_n = \sqrt{\sigma_{n,0}^2 + s ^2}
$$
This will just be one parameter in the model. thejoker also has some default priors for the other orbital paramters:
$$
p(P) \propto \frac{1}{P}; \ P:(P_{min}, P_{max})
$$
$$
p(e) = B(\alpha_e, b_e)
$$
$$
p(\omega) = \mathcal{U}(0, 2\pi)
$$
$$
p(M_0) = \mathcal{U}(0, 2\pi)
$$
$$
p(K) = \mathcal{N}(K|\mu_K, \sigma_K)
$$
$$
\sigma_K = \sigma_{K, 0} \left( \frac{P}{P_0}^{-1/3} \right) (1-e^2)^{-1/2}
$$
$$
p(v_0) = \mathcal{N}(v_0|\mu_{v_0}, \sigma_{v_0})
$$
So, in the above we are fitting the RV curve for the period ($P$), eccentricity ($e$), the pericenter phase ($M_0$) and argument ($\omega$), velocity semi-amplite ($K$) and barycenter velocity ($v_0$).

With the above in mind, below I will demonstrate how to set up the priors (including these default priors). Also, we will intially do rejection sampling of these distriubtions. Essentially, we unformily sample the priors and only accept the parameters if they match the data to some theshold. If enough samples survive this rejection criteria, we have a good estimate of the posterior! If not, we have a good starting point to run the MCMC.

In [ ]:
with pm.Model() as model:
    # Allow extra error to account for under-estimated error bars
    s = xu.with_unit(pm.Lognormal("s", -2, 1), u.km / u.s)

    prior = tj.JokerPrior.default(
        P_min=16 * u.day,
        P_max=128 * u.day,  # Range of periods to consider
        sigma_K0=30 * u.km / u.s,
        P0=1 * u.year,  # scale of the prior on semiamplitude, K
        sigma_v=25 * u.km / u.s,  # std dev of the prior on the systemic velocity, v0
        s=s,
    )

# Generate a large number of prior samples:
prior_samples = prior.sample(size=1_000_000, rng=rnd)

# Run rejection sampling with The Joker:
joker = tj.TheJoker(prior, rng=rnd)
samples = joker.rejection_sample(tres_data, prior_samples, max_posterior_samples=256)

samples

Only one sample survived, but let's see how it looks compared to the Thompson et al. (2019) values:
$$
P = 83.205 \pm 0.064 \ days
$$
$$
e = 0.00476 \pm 0.00255
$$
$$
K = 44.615 \pm 0.123 \ km/s
$$

In [ ]:
samples.tbl["P", "e", "K"]


In [ ]:
_ = tj.plot_rv_curves(samples, data=tres_data)



## Let's add the APOGEE data!

We have more data though! We also have some RV information from APOGEE we can use for the fit. The issue is these two spectrographs have a zero-point offset. So, we will need to add another parameter, which is an offset between the RVs of the two spectrographs. In the below cell, add this as a prior distribution models as a Gaussian with mean of 0 km/s and standard deviation of 5 km/s. Fill in the below code to set this up, run the rejection sample and plot the results!

In [ ]:
data = [apogee_data, tres_data]

with pm.Model() as model:
    # The parameter that represents the constant velocity offset between
    # APOGEE and TRES:
    dv0_1 = ### ADD CODE HERE ###

    # The same extra uncertainty parameter as previously defined
    s = xu.with_unit(pm.Lognormal("s", -2, 1), u.km / u.s)

    # We can restrict the prior on prior now, using the above
    prior_joint = tj.JokerPrior.default(
        # P_min=16*u.day, P_max=128*u.day,
        P_min=75 * u.day,
        P_max=90 * u.day,
        sigma_K0=30 * u.km / u.s,
        P0=1 * u.year,
        sigma_v=25 * u.km / u.s,
        v0_offsets=[dv0_1],
        s=s,
    )

prior_samples_joint = prior_joint.sample(size=1_000_000, rng=rnd)

# Run rejection sampling with The Joker:
joker_joint = tj.TheJoker(prior_joint, rng=rnd)
samples_joint = joker_joint.rejection_sample(
    data, prior_samples_joint, max_posterior_samples=256
)

_ = tj.plot_rv_curves(samples_joint, data=data)



## Run full MCMC!

Let's trying running a full MCMC with this though to visualize the posterior distirubtion. For a Keplarian Orbit, angles are hard for samples to deal with. That is why we will do two things. First, we want to use `pymc3_ext.distributions.angle`, which samples over $cos(x)$ and $sin(x)$ instead. Additionally, we will want to sample over $M_0 - \omega$ and $\omega$, instead of $M_0$ and $\omega$. Then, we will be able to use the rejection sample from the above to initialze the MCMC and run things! Some of the code is filled out by me below, but working on filling in the missing bits based on what I have said above!

In [ ]:
from pymc_ext.distributions import angle

with pm.Model():
    # See note above: when running MCMC, we will sample in the parameters
    # (M0 - omega, omega) instead of (M0, omega)
    M0_m_omega = ### ADD CODE HERE ###
    omega = ### ADD CODE HERE ###
    M0 = ### ADD CODE HERE ###
    # HINT: angle dsitrubtion is simply defined as: angle('omega')

    # The same offset and extra uncertainty parameters as above:
    dv0_1 = xu.with_unit(pm.Normal("dv0_1", 0, 5.0), u.km / u.s)
    s = xu.with_unit(pm.Lognormal("s", -2, 0.5), u.km / u.s)

    prior_mcmc = tj.JokerPrior.default(
        P_min=16 * u.day,
        P_max=128 * u.day,
        sigma_K0=30 * u.km / u.s,
        P0=1 * u.year,
        sigma_v=25 * u.km / u.s,
        v0_offsets=[dv0_1],
        s=s,
        pars={"M0": M0, "omega": omega},
    )

    joker_mcmc = tj.TheJoker(prior_mcmc, rng=rnd)
    mcmc_init = joker_mcmc.setup_mcmc(data, samples_joint)

    trace = pm.sample(
        tune=1000, draws=500, start=mcmc_init, random_seed=seed, cores=1, chains=2
    )


With this done, we can now look at the results!

In [ ]:
# get the samples
mcmc_samples = tj.JokerSamples.from_inference_data(prior_joint, trace, data=data)
mcmc_samples = mcmc_samples.wrap_K()
df = mcmc_samples.tbl.to_pandas()

# look at summary table
az.summary(trace, var_names=prior_mcmc.par_names)


The dataframe above now has all of our samples! Use this and the corner package to plot the resulting MCMC samples

In [ ]:
### ADD CODE HERE ###


Finally, we can look at the binary mass function for the system:
$$
f(M) = \frac{M_2^3 sin^3i}{(M_1 + M_2)^2}
$$
This can accessed for each sample like: `mcmc_samples.get_orbit(i).m_f`. Below, sample 1000 of the random orbits and plot their distirbuton! See how it compares the Thompson et al. (2019); $f(M) = 0.766 \pm 0.00637$

In [ ]:
### ADD CODE HERE ###


So, this is in pretty good agreement and we see that there is a large mass companion!